In [ ]:
import pymongo, itertools, collections
import networkx as nx

In [ ]:
client = pymongo.MongoClient('mongodb://localhost:27017')
players, matches = client['usta'].players, client['usta'].matches
print(players.count(), matches.count())

In [ ]:
def get_total_games(score):
    return sum([int(s[0]) + int(s[1]) for s in score])

def pretty_score(score):
    return ','.join(['-'.join(s) for s in score])

def is_bagel(score):
    return all([not int(s[1]) for s in score])

score = ['76', '64']
print(get_total_games(score))
print(pretty_score(score))
print(is_bagel(score))
print(is_bagel(['60', '30']))

In [ ]:
def get_player_graph(player, prev_player_id=None, graph=None):
    indent = '\t' if prev_player_id is not None else ''
    player_id = int(player['_id'])
    player_full_name = ' '.join([player['first_name'], player['last_name']])
    print('{}{}: {} ({})'.format(
        indent, player_id, player_full_name, player['rating_level']
    ))
    if prev_player_id is None and graph is not None:
        player_rating = player['rating_level'] - 0.25 # start in the middle
        kwargs = dict(name=player_full_name, rating=player_rating)
        graph.add_node(player_id, **kwargs)
    counter = collections.Counter()
    query = {'$or': []}
    # sd_keys, wl_keys = ['singles', 'doubles'], ['winner', 'loser']
    sd_keys, wl_keys = ['singles'], ['winner', 'loser']
    for key in itertools.product(sd_keys, wl_keys):
        query['$or'].append({'.'.join(key): player_id})
    for match in matches.find(query):
        individual_match_found = False
        for sd in sd_keys:
            for individual_match in match[sd]:
                if individual_match_found:
                    break
                for iwl, wl in enumerate(wl_keys):
                    if individual_match[wl] == player_id or (
                        isinstance(individual_match[wl], list) and \
                        player_id in individual_match[wl]
                    ):
                        individual_match_found = True
                        score = individual_match['score']
                        opponent_id = individual_match[wl_keys[int(not(iwl))]]
                        if (
                            get_total_games(score) < 6 or # less than 1 set played
                            is_bagel(score) or # skip bagels
                            opponent_id is None or # default
                            opponent_id == prev_player_id # already added                         
                        ):
                            break
                        counter['{}.{}'.format(sd, wl)] += 1
                        win_or_loss = wl[0].upper()
                        opponent = players.find_one({'_id': str(opponent_id)})
                        opponent_full_name = ' '.join([opponent['first_name'], opponent['last_name']])
                        print(
                            '{}{}:'.format(indent, match['_id']), win_or_loss,
                            pretty_score(score), '\tvs', opponent_full_name, '({})'.format(opponent_id)
                        )
                        if prev_player_id is None: # only go one level down
                            get_player_graph(opponent, prev_player_id=player_id, graph=graph)
                        if graph is not None:
                            opponent_rating = opponent['rating_level'] - 0.25 # start in the middle
                            kwargs = dict(name=opponent_full_name, rating=opponent_rating)
                            graph.add_node(opponent_id, **kwargs)
                        if graph is not None:
                            kwargs = dict(score=score, date=match['date'])
                            if win_or_loss == 'W':
                                graph.add_edge(player_id, opponent_id, **kwargs)
                            else:
                                graph.add_edge(opponent_id, player_id, **kwargs)
                        break
    #print('{}{}'.format(indent, counter))

In [ ]:
G = nx.DiGraph()
player = players.find_one({'last_name': 'Huck', 'first_name': 'Patrick'})
get_player_graph(player, graph=G)
print(G.nodes(data=True))
print(G.edges(data=True))